In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
%pip install lxml


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install html5lib beautifulsoup4 lxml


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pd.read_html("https://tracreports.org/immigration/detentionstats/facilities.html")

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?